In [1]:
nb_users_per_sector  = 5;
nb_sectors           = 6;
nb_RBs               = 10;
total_nb_users       = nb_users_per_sector * nb_sectors;
min_power_per_RB     = 0.1;
max_power_per_sector = 20; 
power_prop_coeff     = 4.7;
power_indep_coeff    = 130;
noise        = float(10)^(-20.4);
bw         = 180e3;

In [2]:
using JuMP

In [3]:
pathloss=rand(nb_users_per_sector,nb_sectors,nb_RBs);

In [4]:
m = Model()
@defVar(m, rate[1:nb_users_per_sector,1:nb_sectors,1:nb_RBs] >= -2)
@defVar(m, poweralloc[1:nb_sectors,1:nb_RBs] >= log(min_power_per_RB))

poweralloc[i,j] ≥ -2.3025850929940455 for all i in {1,2..5,6}, j in {1,2..9,10}

In [5]:
@setNLObjective(m, Max, sum{log(log(exp(rate[i,j,k]+1))), i=1:nb_users_per_sector, j=1:nb_sectors, k=1:nb_RBs})

In [6]:
for j=1:nb_sectors
    @addNLConstraint(m, sum{exp(poweralloc[j,k]), k=1:nb_RBs} <=max_power_per_sector)
end

In [7]:
for i=1:nb_users_per_sector
    for j=1:nb_sectors
        for k=1:nb_RBs
            @addNLConstraint(m, log(exp(rate[i,j,k]-poweralloc[j,k])*noise*bw/pathloss[i,j,k]+sum{exp(rate[i,j,k]+poweralloc[j_p,k]-poweralloc[j,k])*pathloss[i,j_p,k]/pathloss[i,j,k] ,j_p=1:nb_sectors; j_p != j})  <=0)
        end
    end
end

In [8]:
status = solve(m)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     2160
Number of nonzeros in Lagrangian Hessian.............:     8760

Total number of variables............................:      360
                     variables with only lower bounds:      360
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

LoadError: interrupt
while loading In[8], in expression starting on line 1

In [8]:
log_power_sol=getValue(PowerAllocationMatrix);

LoadError: PowerAllocationMatrix not defined
while loading In[8], in expression starting on line 1

In [9]:
power_sol=zeros(nb_sectors,nb_RBs);
for j=1:nb_sectors
    for k=1:nb_RBs
        power_sol[j,k] = exp(log_power_sol[j,k]);
    end
end

LoadError: log_power_sol not defined
while loading In[9], in expression starting on line 2

In [10]:
sum(power_sol,2)

9x1 Array{Float64,2}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0